In [23]:
#********************************************************************
# Note: everything is converted to SI units unless printed otherwise
# an array of integrations for different theta max
#********************************************************************

#Global Constants
kB = 1.38064852*10**(-23) # m^2 kg s^-2 K^-1 (boltzmann constant)
M  = 171*1.66054e-27 # mass of Yb in kg

#Libraries
import matplotlib.pyplot as plt #plotting
import numpy as np
import matplotlib.pylab as pylab #plotting
import scipy.integrate as integrate #integrating
from numpy import sqrt, sin, cos, pi
from decimal import Decimal

#Set Parameters
T_oven       = 450        # temperature of the oven in celcius
len_algn_fud = 3          # length of the triangle alignment fudge factor

#Dimensional Parameters (m)
R_DPT      = 2.5*10**(-3) # radius of DPT
L_DPT      = 518*10**(-3) # length from the end of nozzle to the end of DPT   
trngle_len = np.sqrt(3)*R_DPT

# outer diameter radius and inner array parameters picked from the sheet
rOD = np.array([0.0425,0.0355,0.032,0.028,0.025,0.020,0.0163,0.008])*(0.0254/2) #conversion to meters and radius
rID = np.array([0.035 ,0.031 ,0.029,0.024,0.020,0.016,0.0120,0.006])*(0.0254/2) #conversion to meters and radius

# Convert set temperature in Celcius to Kelvin
T = T_oven + 273

# Pressure, velocities, densities
P        = 10**(5.006 + 9.111 - 8111/T -1.0849*np.log10(T)) #pressure calculation
v_tilde  = np.sqrt(kB*T/M)                                  # convenient redefinition
n_0      = P/(kB*T)                                         # Number of atoms per Volume
v_mp     = np.sqrt(2)*v_tilde                               # most probable velocity
v_avg    = np.sqrt(8/(np.pi))*v_tilde                       # avergae velocity
v_rms    = np.sqrt(3)*v_tilde                               # root-mean-square velocity

# Calculating Dimensional Parameters
N_tubes      = (trngle_len/(2*rOD))*((trngle_len/2*rOD)+1) # number of tubes
A_trngle     = (trngle_len**(2)*sqrt(3)/4)                 # area of triangle nozzle without tubes
A_open       = A_trngle - N_tubes*pi*(rOD**2 - rID**2)     # m^2, total open area of nozzle    
theta_DPT    = np.arctan(R_DPT/L_DPT)                      
L_opt        = rID/(np.tan(theta_DPT))
L_opt_fud    = L_opt/len_algn_fud

# Area Ratio of theta_DPT and theta_mx
#r_sld_angles     = (theta_DPT)**2/(theta_mx)**2 
#r_sld_angles_fud = (theta_DPT)**2/(theta_mx_fud)**2

# Find a theta_mx for each corresponding radius
theta_mx_arr      = np.array([]) # create an array of theta_mx
theta_mx_arr_fud  = np.array([]) # create an array of theta_mx fudges
for i in range(0,len(rID)):
    theta_mx         = np.arctan(rID[i]/L_opt[i])
    theta_mx_fud     = np.arctan(rID[i]/L_opt_fud[i])
    theta_mx_arr     = np.append(theta_mx_arr, theta_mx)
    theta_mx_arr_fud = np.append(theta_mx_arr_fud, theta_mx_fud)

# Integration for each theta_mx
I_result = np.array([])
for i in range (0,len(rID)):
    I        = integrate.quad(lambda x: sin(x)*cos(x)/2, 0, theta_mx_arr[i])
    I_elmnt  = I[0]
    I_result = np.append(I_result, I_elmnt) 
    
# Integration for each theta_mx fudged
I_result_fud = np.array([])
for i in range (0,len(rID)):
    I_fud        = integrate.quad(lambda x: sin(x)*cos(x)/2, 0, theta_mx_arr_fud[i])
    I_elmnt_fud  = I_fud[0]
    I_result_fud = np.append(I_result_fud, I_elmnt_fud) 
    
# Flux calculations
Flux     = v_mp*n_0*A_open*I_result      # flux out of nozzle
Flux_fud = v_mp*n_0*A_open*I_result_fud  # flux out of nozzle fudged
print(theta_DPT)
print(theta_mx_arr)
print(theta_mx_arr_fud)


# Flux after the DPT
#print('Flux: without the fudge (out of DPT)')
#print('%.2E' % Decimal(Flux_DPT))
#print('          ')
#print('Flux: with the fudge (out of DPT)')
#print('%.2E' % Decimal(Flux_DPT_fud))


0.00482621735455257
[0.00482622 0.00482622 0.00482622 0.00482622 0.00482622 0.00482622
 0.00482622 0.00482622]
[0.01447775 0.01447775 0.01447775 0.01447775 0.01447775 0.01447775
 0.01447775 0.01447775]
